# RDF-centered conversion
This notebook will convert ZIB triples to capacity fields.

In [1]:
from rdflib import Graph, URIRef, Namespace
from rdflib.namespace import RDF
from pathlib import Path

## ZIB ontology
A small sample ontology to be able to describe ZIB concepts.

This ontology contains the following ZIB classes.

| Name          | Description  |
|---------------|--------------|
| [MedicationUse](https://zibs.nl/wiki/MedicationUse2-v1.1.1(2020EN)) |    |
| [PharmaceuticalProduct](https://zibs.nl/wiki/PharmaceuticalProduct-v2.1.2(2020EN)) | Partial information model used in MedicationUse |



In [2]:
ZIB = Namespace('http://example.org/ZIB')
zib_ontology = '../../ZIB/zib.owl'
zib_ontology = Graph(identifier=ZIB).parse(zib_ontology)

print(zib_ontology.serialize(format='turtle').decode())

@prefix : <http://example.org/ZIB#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://example.org/ZIB> a owl:Ontology .

:MedicationUse a owl:Class ;
    rdfs:subClassOf :ZibConcept ;
    owl:disjointWith :PharmaceuticalProduct .

:hasZibRecord a owl:FunctionalProperty,
        owl:InverseFunctionalProperty,
        owl:ObjectProperty ;
    rdfs:subPropertyOf :zibProperty .

:medicationCode a owl:FunctionalProperty,
        owl:InverseFunctionalProperty,
        owl:ObjectProperty ;
    rdfs:subPropertyOf :zibProperty .

:PharmaceuticalProduct a owl:Class ;
    rdfs:subClassOf :ZibConcept .

:ZibConcept a owl:Class .

:zibProperty a owl:ObjectProperty ;
    rdfs:subPropertyOf owl:topObjectProperty .




# Sample records
Using the ZIB ontology some sample records are created for patient Bob which uses a medication called *quinidine* which has *C01BA01* within the ATC codesystem.

In [3]:
# Namespaces
ZIB = Namespace('http://example.org/ZIB')
ATC = Namespace('http://purl.bioontology.org/ontology/ATC')

patient = Namespace('http://example.org/patient/')
zib_record = Namespace('http://example.org/zib_record/')

# Sample medicationUse zib
patient_graph = Graph()
patient_graph.bind('ZIB', ZIB)
patient_graph.bind('ATC', ATC)


patient_graph.add((patient.bob, ZIB.hasZibRecord, zib_record.bobsmedication))
patient_graph.add((zib_record.bobsmedication, RDF.type, ZIB.MedicationUse))
patient_graph.add((zib_record.bobsmedication, ZIB.medicationCode, ATC.C01BA01))

print(patient_graph.serialize(format='turtle').decode())

@prefix ATC: <http://purl.bioontology.org/ontology/ATC> .
@prefix ZIB: <http://example.org/ZIB> .

<http://example.org/patient/bob> ZIB:hasZibRecord <http://example.org/zib_record/bobsmedication> .

<http://example.org/zib_record/bobsmedication> a ZIB:MedicationUse ;
    ZIB:medicationCode ATC:C01BA01 .




# Adding ATC ontology


In [5]:
from zib_uploader.tools import load

atc_path = 'atc.ttl'
atc_url = 'http://data.bioontology.org/ontologies/ATC/submissions/12/download?apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb'


with load(atc_url) as atc_file:
    display(atc_file.readlines())

['\n',
 '@prefix skos: <http://www.w3.org/2004/02/skos/core#> .\n',
 '@prefix owl:  <http://www.w3.org/2002/07/owl#> .\n',
 '@prefix rdfs:  <http://www.w3.org/2000/01/rdf-schema#> .\n',
 '@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n',
 '@prefix umls: <http://bioportal.bioontology.org/ontologies/umls/> .\n',
 '\n',
 '\n',
 '<http://purl.bioontology.org/ontology/UATC/>\n',
 '    a owl:Ontology ;\n',
 '    rdfs:comment "RDF Version of the UMLS ontology ATC; converted with the UMLS2RDF tool (https://github.com/ncbo/umls2rdf), developed by the NCBO project." ;\n',
 '    rdfs:label "ATC" ;\n',
 '    owl:imports <http://www.w3.org/2004/02/skos/core> ;\n',
 '    owl:versionInfo "2019ab" .\n',
 '\n',
 '<http://purl.bioontology.org/ontology/UATC/C07CA> a owl:Class ;\n',
 '\tskos:prefLabel """Beta blocking agents, non-selective, and other diuretics"""@en ;\n',
 '\tskos:notation """C07CA"""^^xsd:string ;\n',
 '\trdfs:subClassOf <http://purl.bioontology.org/ontology/UATC/C07C> ;\n',
 '\t<ht